# 1: Candidate Extraction

In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['SNORKELDB']="postgres://jhusson@localhost:5432/snorkel_strom"

from snorkel import SnorkelSession
session = SnorkelSession()

## Loading the `Sentence` objects

In [2]:
from snorkel.models import Sentence

sentences = session.query(Sentence).limit(10000).all()

## Defining a `Candidate` schema
We now define the schema of the relation mention we want to extract (which is also the schema of the candidates).  This must be a subclass of `Candidate`, and we define it using a helper function.

Here we'll define a binary _spouse relation mention_ which connects two `Span` objects of text.  Note that this function will create the table in the database backend if it does not exist:

In [3]:
from snorkel.models import candidate_subclass

StromStrat = candidate_subclass('StromStrat', ['strom', 'stratname'])

## Writing a basic `CandidateExtractor`

Next, we'll write a basic function to extract **candidate spouse relation mentions** from the corpus.  The `SentenceParser` we used in Part I is built on [CoreNLP](http://stanfordnlp.github.io/CoreNLP/), which performs _named entity recognition_ for us.

We will extract `Candidate` objects of the `Spouse` type by identifying, for each `Sentence`, all pairs of ngrams (up to trigrams) that were tagged as people.

First, we define a child context space for our sentences.

In [4]:
from snorkel.candidates import Ngrams

ngram_strom = Ngrams(n_max=1)
ngram_strat = Ngrams(n_max=9)

Next, we use a `PersonMatcher` to enforce that candidate relations are composed of pairs of spans that were tagged as people by the `SentenceParser`.

In [5]:
from snorkel.matchers import RegexMatchSpan

strom_matcher = RegexMatchSpan(rgx="stromatolit|thrombolit")

In [6]:
from snorkel.matchers import DictionaryMatch
import urllib
import json

request = urllib.urlopen('https://macrostrat.org/api/v2/defs/strat_names?all')
data = json.loads(request.read())

#FULL STRAT NAME
strat_dict_long = { r['strat_name_long'] for r in data['success']['data'] }

#ABBREVIATED STRAT NAME - V1
strat_dict_abV1 = { r['strat_name'] + ' ' + r['rank'] for r in data['success']['data'] }

#ABBREVIATED STRAT NAME - V2
strat_dict_abV2 = { r['strat_name'] + ' ' + r['rank'] + '.' for r in data['success']['data'] }

#LITHOLOGY STRAT NAMES
request = urllib.urlopen('https://macrostrat.org/api/v2/defs/lithologies?all')
lithologies = json.loads(request.read())
lithologies=[l['name'].capitalize() for l in lithologies['success']['data']]

strat_dict_short = { r['strat_name'] for r in data['success']['data'] }

strat_dict_lith=set()
for r in strat_dict_short:
    if r.split(' ')[-1] in lithologies:
        strat_dict_lith.add(r)
        
strat_dict=set(list(strat_dict_long)+list(strat_dict_abV1)+list(strat_dict_abV2)+list(strat_dict_lith))
        
strat_matcher=DictionaryMatch(d=strat_dict,ignore_case=False,longest_match_only=True)


Finally, we combine the candidate class, child context space, and matcher into an extractor.

In [7]:
from snorkel.candidates import CandidateExtractor

ce = CandidateExtractor(StromStrat, [ngram_strom, ngram_strat], [strom_matcher, strat_matcher],
                        symmetric_relations=True, nested_relations=False, self_relations=False)

## Running the `CandidateExtractor`

We run the `CandidateExtractor` by calling extract with the contexts to extract from, a name for the `CandidateSet` that will contain the results, and the current session.

In [8]:
%time c = ce.extract(sentences, 'Candidate Set', session)
print "Number of candidates:", len(c)

[========================================] 100%                                       ] 0%

CPU times: user 46.4 s, sys: 5.86 s, total: 52.3 s
Wall time: 1min 1s
Number of candidates: 36


### Saving the extracted candidates

In [9]:
session.add(c)
session.commit()

### Reloading the candidates

In [16]:
from snorkel.models import CandidateSet
c = session.query(CandidateSet).filter(CandidateSet.name == 'Candidate Set').one()
c=c.candidates[-5]
print c

StromStrat(Span("stromatolites", parent=12144, chars=[117,129], words=[18,18]), Span("Rae Group", parent=12144, chars=[138,146], words=[21,22]))


## Using the `Viewer` to inspect candidates

Next, we'll use the `Viewer` class--here, specifically, the `SentenceNgramViewer`--to inspect the data.

It is important to note, our goal here is to **maximize the recall of true candidates** extracted, **not** to extract _only_ the correct candidates. Learning to distinguish true candidates from false candidates is covered in Tutorial 4.

First, we instantiate the `Viewer` object, which groups the input `Candidate` objects by `Sentence`:

In [24]:
from snorkel.viewer import SentenceNgramViewer

# NOTE: This if-then statement is only to avoid opening the viewer during automated testing of this notebook
# You should ignore this!
import os
if 'CI' not in os.environ:
    sv = SentenceNgramViewer(c, session, annotator_name="Tutorial Part 2 User")
else:
    sv = None

<IPython.core.display.Javascript object>

Next, we render the `Viewer.

In [25]:
sv

Note that we can **navigate using the provided buttons**, or **using the keyboard (hover over buttons to see controls)**, highlight candidates (even if they overlap), and also **apply binary labels** (more on where to use this later!).  In particular, note that **the Viewer is synced dynamically with the notebook**, so that we can for example get the `Candidate` that is currently selected. Try it out!

In [30]:
if 'CI' not in os.environ:
    print unicode(sv.get_selected())
    print sv.get_selected()[0].parent.id

StromStrat(Span("stromatolitic", parent=11957, chars=[16,28], words=[3,3]), Span("Glenelg Formation", parent=11957, chars=[68,84], words=[12,13]))
11957


### Repeating for development and test corpora
We will rerun the same operations for the other two news corpora: development and test. All we do for each is load in the `Corpus` object, collect the `Sentence` objects, and run them through the `CandidateExtractor`.

In [ ]:
for corpus_name in ['News Development', 'News Test']:
    corpus = session.query(Corpus).filter(Corpus.name == corpus_name).one()
    sentences = set()
    for document in corpus:
        for sentence in document.sentences:
            if number_of_people(sentence) < 5:
                sentences.add(sentence)
    
    %time c = ce.extract(sentences, corpus_name + ' Candidates', session)
    session.add(c)
session.commit()

Next, in Part 3, we will annotate some candidates with labels so that we can evaluate performance.